# Подключение библиотек

In [1]:
pip install pytelegrambotapi

     |████████████████████████████████| 147 kB 24.8 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.4.0-py3-none-any.whl size=128152 sha256=8c59b7adf5ceb46b46716f3378184eb552a180127d6b3b38b3ccdfd09daa0225
  Stored in directory: /root/.cache/pip/wheels/1e/6d/ff/435b12799b8212017f08dbbfdb7a1e5174d72d20fb7c0f4703
Successfully built pytelegrambotapi


In [2]:
import telebot
import os
import time
import logging
from telebot import apihelper
import requests
import pprint
from telebot import types 
import random 
import pandas as pd
import xlrd
import datetime
import openpyxl
from csv import writer
import numpy as np

In [4]:
from google.colab import files 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Интегрируем данные

In [5]:
sticer= '/content/drive/My Drive/Базы/OblakoVShtanah_004.webp'
direct='/content/drive/My Drive/Базы/'
path2=direct+'Рекомендации по развитию.csv'
path3=direct+'person_database.csv'

In [25]:
#column_path3=['ID','PATH']
#with open(path3, 'a',newline='') as tab3:
#  obj=writer(tab3)
#  obj.writerow(column_path3)
#  tab3.close()

In [9]:
list_input=['расход','Расход', 'Доход','доход', ]
list_output=["Получить", "Дать", "Проанализируй", "получить", "дать", "проанализируй", "Узнать", "Показать", "узнать", "показать"]

# Базовые функции обработки сообщений

In [10]:
def create_person_file(name):
  if name in os.listdir(direct):
    path=direct+name+'.csv'
  else:
    path=direct + name +'.csv'
    column=['Направление бюджета','Статья бюджета','Сумма','Полная дата','Месяц']
    file_one=open(path,'a')
    writer_place=writer(file_one)
    writer_place.writerow(column)
    file_one.close()
  return path

In [11]:
def check(number,path): 
  path_list=pd.read_csv(path3)
  info=[number,path]
  if len(path_list) >= 1:
    if path in path_list:
      null_pac='0'
    else:
      with open(path3, 'a', newline='') as tab3:
        obj=writer(tab3)
        obj.writerow(info)
        tab3.close()
  elif len(path_list) == 0:
    with open(path3, 'a',newline='') as tab3:
      obj=writer(tab3)
      obj.writerow(info)
      tab3.close()

In [12]:
def get_id(number):
  list_person=pd.read_csv(path3)
  person_id = list_person.loc[list_person['ID'] == number]
  person_path = person_id['PATH'][0]
  return person_path

In [13]:
def get_data(text,path):
  for_answer=text.split(',')
  full_date=datetime.datetime.now()
  only_month=full_date.strftime("%B")
  for_answer.append(full_date)
  for_answer.append(only_month)
  with open(path, 'a', newline='') as f:
    writer_obj=writer(f)
    writer_obj.writerow(for_answer)
    f.close()
  return for_answer[0]

In [14]:
def loading_data(srez, month, path): 
  new_file=pd.read_csv(path) 
  if len(new_file) >= 1:
    if srez == 'Месячный':
      month_table=new_file.loc[new_file['Месяц'] == month]
      plus=month_table.loc[month_table['Направление бюджета'] == 'доход']
      volume_plus=sum(plus['Сумма']) 
      minus=month_table.loc[month_table['Направление бюджета'] == 'расход']
      volume_minus=sum(minus['Сумма'])
      proc_minus=int(volume_minus*100/volume_plus) 
      if len(plus) and len(minus) > 1:
        volume_max=max(minus['Сумма']) #!!!
        max_minus=minus.loc[minus['Сумма'] == volume_max] #!!!
        line_max=max_minus.iloc[0][1] #!!!
        output = str('Данные за ') + str(month) + str(': сумма доходов в этом месяце составляет ') + str(volume_plus) +str(' руб.; сумма расходов в этом месяце составляет ') + str(volume_minus) + str(' руб.; доля расходов в месячном бюджете составляет ') + str(proc_minus) + str(' %; максимальный расход в этом месяце связан с покупкой "') + str(line_max) + str('" в размере ') + str(volume_max) + str(' руб.')
      else:
        output = str('Данные за ') + str(month) + str(': сумма доходов в этом месяце составляет ') + str(volume_plus) +str(' руб.; сумма расходов в этом месяце составляет ') + str(volume_minus) + str(' руб.; доля расходов в месячном бюджете составляет ') + str(proc_minus) + str(' %.\n\nСо временем, при увеличении объема данных в базе, ты сможешь получать более полную и детальную информацию о своём бюджете.') 
    if srez != 'Месячный':
      plus=new_file.loc[new_file['Направление бюджета'] == 'доход']
      volume_plus=sum(plus['Сумма'])  
      minus=new_file.loc[new_file['Направление бюджета'] == 'расход']
      volume_minus=sum(minus['Сумма']) 
      potenc_fond=volume_plus-volume_minus
      mean_plus = plus['Сумма'].mean()
      mean_minus = minus['Сумма'].mean()
      proc_mean_minus = mean_minus*100/potenc_fond
      proc_mean_plus =  mean_minus*100/mean_plus
      uniqum_month=list(set(new_file['Месяц']))
      len_listmonth=len(uniqum_month)
      if len_listmonth == 1:
        name_period=' месяц:'
      elif len_listmonth == len_listmonth in range(2,5):
        name_period=' месяца:'
      else:
        name_period=' месяцев:'
      if potenc_fond < 0 and proc_mean_minus > 20: 
        conclusion='существенном дефиците бюджета. Рекоммендую экономить. Если еще не знаешь как, кликай по ссылке ниже:\n\nhttps://www.udemy.com/course/drxvusyu/' 
      elif potenc_fond < 0 and proc_mean_minus < 20:
        conclusion='дефиците бюджета'
      elif potenc_fond > 0 and proc_mean_plus < 20:
        conclusion='существенном префиците бюджета. Рекоммендую инвестировать. Если еще не знаешь куда, кликай по ссылке ниже:\n\nhttps://school.moex.com/put-investora/' 
      else:
        conclusion='префиците бюджета'
      output = str('В базе представлены данные за ') + str(len_listmonth) + str(name_period) + str(' сумма доходов за весь период составляет ') + str(volume_plus) + str(' руб.; сумма расходов за весь период составляет ') + str(volume_minus)+str(' руб.; объем резервного фонда за весь период составляет ') + str(potenc_fond) + str(' руб., что свидетельствует о ') + str(conclusion)
  else:
    output = str("Данные в базе не найдены. Введи данные о своем бюджете и попробуй снова")
  report = output
  return report

In [15]:
def recomendation_list(text):
  recomend=text.split(',')
  with open(path2, 'a', newline='') as rec: 
    writer_object=writer(rec)
    writer_object.writerow(recomend)
    rec.close()
  return str('Я пока этого делать не умею, но вскоре, с твоей помощью, я буду уметь анализировать и это.\n\nЯ занес твой запрос в лист')

# Конфигурация бота

In [16]:
TOKEN = '5098309304:AAFLVJ4GRLnKlAPWeHunheVSeGuXhT7X77s'
url_bot=f'https://api.telegram.org/bot{TOKEN}'
proxies={'http': 'russia-dd.proxy.digitalresistance.dog'}
url=f'{url_bot}/getMe'
resuls=requests.get(url, proxies=proxies)
print(resuls.status_code)

200


In [22]:
apihelper.proxy = proxies
bot = telebot.TeleBot(TOKEN)

In [24]:
@bot.message_handler(commands = ['start'])
def commnad_start_function(message):
  number=message.from_user.id
  path=create_person_file(str(number))
  check(number,path)
  bot.reply_to(message, 'Я твой финансовый помощник и я рад, что ты доверился мне!\n\nНажми на /help, чтобы ознакомиться с моими возможностями и методикой. Если ты обо мне уже все знаешь, можем смело приступать. Для начала тыкни на /input для ввода первых значений и ознакомления с краткой инструкцией по вводу дальнейших данных')
  sticker = open(sticer, 'rb')
  bot.send_sticker(message.chat.id, sticker)

@bot.message_handler(commands = ['help'])
def commnad_help_function(message):
  bot.reply_to(message, 'Тебе больше не нужно держать в голове все свои финансы и вести реестр бюджета. От тебя мне нужны только данные, остальное я буду делать за тебя - запоминать, суммировать, классифицировать, а когда тебе понадобится, выведу на экран месячную или сводную статистику по бюджету с помощью команды /analasis или предоставлю тебе сведения по твоему персональному запросу. Для этого можно использовать, например, следующие ключевые слова: "Получить", "Узнать", "Показать". При этом реестр не имеет значения, я смогу понять тебя, а вот грамматика важна. И помни: прежде чем что-то получить, нужно что-то внести. Я к твоим услугам, удачи в использовании.')

@bot.message_handler(commands = ['input'])
def commnad_input_date_function(message):
	bot.reply_to(message, 'Напиши через сообщение в формате "направление бюджета, статья бюджета и сумма", разделив каждое значение запятой.\n\nНАПРИМЕР: доход, аванс, 10000\nИЛИ: расход, оплата ж/к услуг, 4500')

@bot.message_handler(commands = ['analasis'])
def commnad_output_date_function(message):
	inMurkup2 = types.InlineKeyboardMarkup(row_width=1)
	but1 = types.InlineKeyboardButton("Месячный",callback_data='but1')
	but2 = types.InlineKeyboardButton("Сводный", callback_data='but2')
	inMurkup2.add(but1, but2)
	bot.send_message(message.chat.id, "{0.first_name}, за какой период необходимо предоставить данные?".format(message.from_user),parse_mode='html',reply_markup=inMurkup2)
 
@bot.callback_query_handler(func=lambda call: True)   
def callback_input(call):
  try:
    if call.message:
      if call.data == 'but1':
        number=call.from_user.id
        path=get_id(number)
        month_report=datetime.datetime.now().strftime("%B")
        srez='Месячный'
        report=loading_data(srez,month_report,path)
        bot.send_message(call.message.chat.id, report)
      elif call.data == 'but2':
        number=call.from_user.id
        path=get_id(number)
        month_report=datetime.datetime.now().strftime("%B")
        srez='Сводный'
        report=loading_data(srez,month_report,path)
        bot.send_message(call.message.chat.id, report)
      bot.edit_message_text(chat_id=call.message.chat.id, message_id=call.message.message_id, text="Что мы имеем!", reply_markup=None)
  except Exception as e:
    print(repr(e))


@bot.message_handler(content_types = ['text'])
def recieve_text(message):
  number=message.from_user.id
  path=get_id(number)
  text = message.text
  if any(n in text for n in list_input):
    answer = get_data(text,path) 
    bot.reply_to(message, 'Данные по ' + answer + 'y' + ' внесены в реестр.\n\nОни всегде доступны для вывода на экран с помощью команды /analasis или использования ключевых слов (примеры приведены в /help')  
  elif any(n in text for n in list_output):
    month_report=datetime.datetime.now().strftime("%B")
    srez='Сводный'
    report=loading_data(srez,month_report,path)
    print()
    bot.reply_to(message, report)
  else:
    answer = recomendation_list(text)
    bot.reply_to(message, answer)




bot.polling(none_stop=True) 